### Pruebas de las librerias

In [42]:
import os, math, emerge_classes
from dynamixel_sdk import *                    # Uses Dynamixel SDK library
from joint_handler import *
from zmqRemoteApi import RemoteAPIClient

from random import randint as ri
from random import uniform as ru

### Pruebas para robot

In [43]:
isSim = False

In [44]:
if isSim:
    cluster = False
    setup = Sim_setup(cluster)
    client = RemoteAPIClient('localhost', setup.port_conection)
    sim = client.getObject('sim')
    jointHandler = JointHandler(isSim, client, sim)
else:
    setup = EMERGE_setup()
    portHandler = PortHandler(setup.DEVICENAME)
    packetHandler = PacketHandler(setup.PROTOCOL_VERSION)
    jointHandler = JointHandler(isSim, portHandler, packetHandler)

In [45]:
jointHandler.connectEMERGE()

Succeeded to open the port
Succeeded to change the baudrate


In [6]:
jointHandler.enableJointMovement(1)

In [7]:
jointHandler.getJointStatus(1)
jointHandler.getJointStatus(2)

[ID:001] ping Succeeded.


False

In [ ]:
jointHandler.num_joints,jointHandler.max_num_joint

In [ ]:
jointHandler.joint_ids = []
jointHandler.num_joints = 0

In [ ]:
jointHandler.getJointList()

In [ ]:
print(f"Hay {jointHandler.num_joints} y son {jointHandler.joint_ids}")

In [9]:
jointHandler.disconnectEMERGE()

In [ ]:
jointHandler.loadEMERGE()

In [ ]:
jointHandler.num_joints,jointHandler.joint_ids

In [ ]:
jointHandler.exact_rad

In [ ]:
test_pos = [-math.pi/4,0,math.pi/4,0]
for pos in test_pos:
    jointHandler.setJointTargetPosition(1,pos)
    error = abs(pos - jointHandler.getJointPosition(1))
    print(f"Error: {error*189/math.pi}")

In [ ]:
jointHandler.joint_ids = []
jointHandler.getJointList()

In [ ]:
jointHandler.joint_ids

In [ ]:
jointHandler.unloadEMERGE()

#### Prueba de casi todas las funciones para robot

In [46]:
jointHandler.loadEMERGE()

Succeeded to open the port
Succeeded to change the baudrate
[ID:001] ping Succeeded.
[ID:012] ping Succeeded.


In [47]:
jointHandler.joint_ids

[1, 12]

In [48]:
test_pos = [-math.pi/4,0,math.pi/4,0]
for pos in test_pos:
    jointHandler.setJointTargetPosition(1,pos)
    error = abs(pos - jointHandler.getJointPosition(1))
    print(f"Error: {error*189/math.pi}")

Error: 1.0458984374999942
Error: 0.9228515625000169
Error: 0.12304687500000404
Error: 0.9228515624999902


In [ ]:
jointHandler.unloadEMERGE()

### Pruebas para simulacion

In [3]:
isSim = True
if isSim:
    cluster = False
    setup = Sim_setup(cluster)
    client = RemoteAPIClient('localhost', setup.port_conection)
    sim = client.getObject('sim')
    jointHandler = JointHandler(isSim, client, sim)
else:
    setup = EMERGE_setup()
    portHandler = PortHandler(setup.DEVICENAME)
    packetHandler = PacketHandler(setup.PROTOCOL_VERSION)
    jointHandler = JointHandler(isSim, portHandler, packetHandler)

In [4]:
jointHandler.loadEMERGE()

In [6]:
print(jointHandler.__dict__)

{'client': <zmqRemoteApi.RemoteAPIClient object at 0x000001144FF6B6D0>, 'sim': <class 'zmqRemoteApi.sim'>, 'exact_rad': 0.017453292519943295, 'joint_ids': [18, 22, 26], 'num_joints': 3, 'obj_ids': [28]}


In [10]:
initial_pos = [-math.pi/4, math.pi/4, -math.pi/4]
#Set all the joints to their initial position
for joint_n in range(jointHandler.num_joints):
    init_pos = initial_pos[joint_n]
    joint = jointHandler.joint_ids[joint_n]

    jointHandler.setJointInitialPosition(joint,init_pos)
    print(jointHandler.getJointPosition(joint))


-0.7853981852531433
0.7853981852531433
-0.7853981852531433


In [ ]:
for joint_n in range(jointHandler.num_joints):
    init_pos = initial_pos[joint_n]
    joint = jointHandler.joint_ids[joint_n]

    jointHandler.setJointInitialPosition(joint,init_pos)
    print(jointHandler.getJointPosition(joint))

In [12]:
#Generate a list for random starting positions
def set_starting_position ():
    starting_pos = []
    starting_angles = [0, 45, 75]
    angle = ri(0, len(starting_angles)-1)
    sentido = math.pow(-1,ri(0, 1))
    for _ in jointHandler.joint_ids:
        starting_pos.append(sentido*starting_angles[angle]*math.pi/180)
    return starting_pos


def init_position():
    initial_pos = set_starting_position()
    #Set all the joints to their initial position
    for joint_n in range(jointHandler.num_joints):
        init_pos = initial_pos[joint_n]
        joint = jointHandler.joint_ids[joint_n]

        jointHandler.setJointInitialPosition(joint,init_pos)
        print(jointHandler.getJointPosition(joint))

In [15]:
init_position()

-0.7853981852531433
-0.7853981852531433
-0.7853981852531433


In [18]:
#Class to store all the perceptions
class perceptions ():
    def __init__(self):
        #Execution data
        self.sequence = 0
        self.step = 0
        self.increments = []
        #Initial perceptions
        self.prev_j_positions = []
        self.prev_pos_x = 0
        self.prev_pos_y = 0
        self.prev_pos_z = 0
        #Resulting perceptions
        self.post_j_positions = []
        self.post_pos_x = 0
        self.post_pos_y = 0
        self.post_pos_z = 0

perception = perceptions()

In [35]:
def rand_gen():
    for num in range(jointHandler.num_joints):
        n = ru(-80, 85) * math.pi / 180
        perception.increments.append(n)

In [16]:
jointHandler.connectEMERGE()

In [38]:
perception = perceptions()
rand_gen()

In [39]:
perception.increments

[0.45237746982119986, -0.8993947895509815, 0.012890266707569173]

In [33]:
init_position()

1.3089969158172607
1.3089969158172607
1.3089969158172607


In [40]:
for joint_n in range(jointHandler.num_joints):
    next_pos = perception.increments[joint_n]
    joint = jointHandler.joint_ids[joint_n]

    jointHandler.setJointTargetPosition(joint,next_pos)
    print(jointHandler.getJointPosition(joint))

0.4470522403717041
-0.9167826175689697
0.004396915435791016


In [41]:
jointHandler.disconnectEMERGE()